In [36]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [37]:
# 패키지 설치
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, precision_score, recall_score, auc
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, f1_score

from bayes_opt import BayesianOptimization

In [38]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [39]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import TomekLinks
from collections import Counter

In [40]:
final = pd.read_csv(r'C:\Users\new\OneDrive\바탕 화면\data\final.csv')
final.head()

,user_id,sum_applied,application_id,gender,insert_time,yearly_income,income_type,employment_type,houseown_type,desired_amount,...,is_applied,weekday,loanapply_insert_hour,bank_apply_rate,product_apply_rate,cluster2,action_cnt,use_day_cnt,first_event,timeout_sum(s)
0,545882.0,8.0,576643,1.0,2022-05-09 10:54:51,72000000.0,직장가입자(4대보험O),정규직,전월세,20000000.0,...,0.0,0,10,0.042501,0.050284,0,41.913806,5.765978,CompleteIDCertification,67.997282
1,545882.0,8.0,576643,1.0,2022-05-09 10:54:51,72000000.0,직장가입자(4대보험O),정규직,전월세,20000000.0,...,0.0,0,10,0.017018,0.016149,0,41.913806,5.765978,CompleteIDCertification,67.997282
2,558819.0,5.0,2136706,1.0,2022-05-09 10:41:05,39000000.0,직장가입자(4대보험O),정규직,전월세,80000000.0,...,0.0,0,10,0.024636,0.028622,0,41.913806,5.765978,CompleteIDCertification,67.997282
3,558819.0,5.0,2136706,1.0,2022-05-09 10:41:05,39000000.0,직장가입자(4대보험O),정규직,전월세,80000000.0,...,0.0,0,10,0.021634,0.035367,0,41.913806,5.765978,CompleteIDCertification,67.997282
4,558819.0,5.0,2136706,1.0,2022-05-09 10:41:05,39000000.0,직장가입자(4대보험O),정규직,전월세,80000000.0,...,0.0,0,10,0.027749,0.035265,4,41.913806,5.765978,CompleteIDCertification,67.997282


In [41]:
final = final.drop(['user_id', 'application_id', 'insert_time', 'loanapply_insert_time','loanapply_insert_hour'], axis = 1)

In [42]:
final.isnull().sum()

sum_applied                                  0
gender                                       0
yearly_income                                0
income_type                                  0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
reage                                        0
career                                       0
credit_grade                                 0
spec_clust                                   0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
is_applied                             3255482
weekday      

In [43]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13141813 entries, 0 to 13141812
Data columns (total 29 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   sum_applied                          float64
 1   gender                               float64
 2   yearly_income                        float64
 3   income_type                          object 
 4   employment_type                      object 
 5   houseown_type                        object 
 6   desired_amount                       float64
 7   purpose                              object 
 8   personal_rehabilitation_yn           float64
 9   personal_rehabilitation_complete_yn  float64
 10  existing_loan_cnt                    float64
 11  existing_loan_amt                    float64
 12  reage                                float64
 13  career                               float64
 14  credit_grade                         float64
 15  spec_clust                    

In [44]:
final['income_type'] = final['income_type'].astype('category')
final['employment_type'] = final['employment_type'].astype('category')
final['houseown_type'] = final['houseown_type'].astype('category')
final['purpose'] = final['purpose'].astype('category')
final['first_event'] = final['first_event'].astype('category')
final['weekday'] = final['weekday'].astype('category')
final['personal_rehabilitation_yn'] = final['personal_rehabilitation_yn'].astype('category')
final['personal_rehabilitation_complete_yn'] = final['personal_rehabilitation_complete_yn'].astype('category')
final['gender'] = final['gender'].astype('category')
final['spec_clust'] = final['spec_clust'].astype('category')
final['cluster2'] = final['cluster2'].astype('category')

In [45]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13141813 entries, 0 to 13141812
Data columns (total 29 columns):
 #   Column                               Dtype   
---  ------                               -----   
 0   sum_applied                          float64 
 1   gender                               category
 2   yearly_income                        float64 
 3   income_type                          category
 4   employment_type                      category
 5   houseown_type                        category
 6   desired_amount                       float64 
 7   purpose                              category
 8   personal_rehabilitation_yn           category
 9   personal_rehabilitation_complete_yn  category
 10  existing_loan_cnt                    float64 
 11  existing_loan_amt                    float64 
 12  reage                                float64 
 13  career                               float64 
 14  credit_grade                         float64 
 15  spec_clust   

In [46]:
final = pd.get_dummies(final)
final

,sum_applied,yearly_income,desired_amount,existing_loan_cnt,existing_loan_amt,reage,career,credit_grade,bank_id,product_id,...,cluster2_1,cluster2_2,cluster2_3,cluster2_4,cluster2_5,first_event_CompleteIDCertification,first_event_EndLoanApply,first_event_GetCreditInfo,first_event_StartLoanApply,first_event_others
0,8.0,72000000.0,20000000.0,7.0,106000000.0,40.0,4.0,7.0,54,235,...,0,0,0,0,0,1,0,0,0,0
1,8.0,72000000.0,20000000.0,7.0,106000000.0,40.0,4.0,7.0,11,118,...,0,0,0,0,0,1,0,0,0,0
2,5.0,39000000.0,80000000.0,2.0,58000000.0,40.0,11.0,5.0,42,216,...,0,0,0,0,0,1,0,0,0,0
3,5.0,39000000.0,80000000.0,2.0,58000000.0,40.0,11.0,5.0,25,169,...,0,0,0,0,0,1,0,0,0,0
4,5.0,39000000.0,80000000.0,2.0,58000000.0,40.0,11.0,5.0,22,221,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13141808,4.0,130000000.0,100000000.0,0.0,0.0,40.0,11.0,3.0,60,183,...,0,0,0,0,0,1,0,0,0,0
13141809,14.0,64000000.0,20000000.0,8.0,151000000.0,40.0,3.0,6.0,2,7,...,0,0,0,0,0,1,0,0,0,0
13141810,6.0,58000000.0,3000000.0,1.0,43500000.0,40.0,16.0,7.0,59,150,...,0,0,0,0,0,1,0,0,0,0
13141811,2.0,35000000.0,6000000.0,3.0,27000000.0,40.0,2.0,7.0,49,136,...,0,0,0,0,0,1,0,0,0,0


In [47]:
final_train = final[final['is_applied'].notnull()]
final_test = final[final['is_applied'].isnull()]

In [48]:
data = final_train.drop('is_applied',axis=1)
target = final_train['is_applied']

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data, target, test_size=0.3, random_state=0)

# UnderSam + XGB

In [50]:
undersample = RandomUnderSampler(sampling_strategy='majority')
X_rus, y_rus = undersample.fit_resample(X_train, y_train)
print(Counter(y_rus))

Counter({0.0: 392199, 1.0: 392199})


In [52]:
xgb_rus = XGBClassifier()
xgb_rus.fit(X_rus, y_rus)
xgb_rus_preds= xgb_rus.predict(X_val)

In [55]:
#f1 score
print('f1 score : {:0.4f}'.format(f1_score(y_val, xgb_rus_preds)))
#정확도
print('accuracy: {:0.4f}'.format(accuracy_score(y_val, xgb_rus_preds)))
# 정밀도
print('precision: {:0.4f}'.format(precision_score(y_val, xgb_rus_preds)))
#재현율
print('recall: {:0.4f}'.format(recall_score(y_val,xgb_rus_preds)))
# ROC_AUC
print('roc_auc: {:0.4f}'.format(roc_auc_score(y_val, xgb_rus_preds)))

f1 score : 0.3115
accuracy: 0.7828
precision: 0.1899
recall: 0.8666
roc_auc: 0.8222


# RandomSearch

In [68]:
from sklearn.model_selection import RandomizedSearchCV

In [82]:
parameters = {'max_depth':[5,10,20], 'gamma' : [0, 0.25, 1.0], 'reg_lambda' : [0,1,10],
             'learning_rate':[0.05,0.1,0.2], 'scale_pos_weight':[5,8,11]}

In [83]:
xgb = XGBClassifier()

In [85]:
from sklearn.model_selection import RandomizedSearchCV
clf=RandomizedSearchCV(xgb,parameters,scoring='precision',n_iter=10)
clf.fit(X=X_train, y=y_train)
print(clf.best_params_)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\new\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_18804\2279282124.py", line 3, in <cell line: 3>
    clf.fit(X=X_train, y=y_train)
  File "C:\Users\new\anaconda3\lib\site-packages\sklearn\model_selection\_search.py", line 875, in fit
    # of the params are estimators as well.
  File "C:\Users\new\anaconda3\lib\site-packages\sklearn\model_selection\_search.py", line 1753, in _run_search
  File "C:\Users\new\anaconda3\lib\site-packages\sklearn\model_selection\_search.py", line 852, in evaluate_candidates
  File "C:\Users\new\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 367, in _warn_or_raise_about_fit_failures
    either binary or multiclass, :class:`StratifiedKFold` is used. In all
ValueError: 
All the 50 fits failed.
It is very likely that your mode

In [ ]:
xgb_param = XGBClassifier(scale_pos_weight = 5, num_leaves = 80, min_child_samples = 5, max_depth = 20, learning_rate = 0.1)
xgb_param.fit(X_train, y_train)
xgb_param_preds= lgb_param.predict(X_train)

In [73]:
rs_model=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='f1',n_jobs=-1,cv=5,verbose=3)

In [74]:
rs_model.fit(X_rus,y_rus)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Unexpected exception formatting exception. Falling back to standard exception


joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\new\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py", line 436, in _process_worker
    r = call_item()
  File "C:\Users\new\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py", line 288, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "C:\Users\new\anaconda3\lib\site-packages\joblib\_parallel_backends.py", line 595, in __call__
    return self.func(*args, **kwargs)
  File "C:\Users\new\anaconda3\lib\site-packages\joblib\parallel.py", line 262, in __call__
    return [func(*args, **kwargs)
  File "C:\Users\new\anaconda3\lib\site-packages\joblib\parallel.py", line 262, in <listcomp>
    return [func(*args, **kwargs)
  File "C:\Users\new\anaconda3\lib\site-packages\sklearn\utils\fixes.py", line 221, in __call__
    with config_context(**self.config):
  File "C:\Users\new\anaconda3\lib\contextlib.py", line 119, in __ente

In [75]:
rs_model.best_estimator_

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [ ]:
xgb_rus1 = XGBClassifier()
xgb_rus.fit(X_rus, y_rus)
xgb_rus_preds= xgb_rus.predict(X_val)

In [57]:
pip install searchgrid

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\new\\anaconda3\\Lib\\site-packages\\~-learn\\.libs\\vcomp140.dll'
Consider using the `--user` option or check the permissions.




  Created wheel for searchgrid: filename=searchgrid-0.2.0-py3-none-any.whl size=5764 sha256=362058e02a4dece3d9b4a6245ee078f24c0012df55bd916b59dd820549b77bcd
  Stored in directory: c:\users\new\appdata\local\pip\cache\wheels\8b\81\7b\671736b19540da04d89682539d4301be8f43f4640bbd0d3223
Successfully built searchgrid
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.2
    Uninstalling scikit-learn-1.1.2:
      Successfully uninstalled scikit-learn-1.1.2


In [63]:
pip install hyperopt

Note: you may need to restart the kernel to use updated packages.


In [64]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [65]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

In [66]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_rus, y_rus), ( X_val, y_val)]
    
    clf.fit(X_rus, y_rus,
            eval_set=evaluation, eval_metric="f1",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_val)
    accuracy = accuracy_score(y_val, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': roc, 'status': STATUS_OK }

In [67]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

job exception: 'XGBClassifier' object has no attribute 'XGBClassifier'



  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]


AttributeError: 'XGBClassifier' object has no attribute 'XGBClassifier'

In [ ]:
xgb= XGBClassifier()

xgb_param_grid={
    'max_depth':[3,4,5],
    'learnig_rate':[1.0,0.5,0.1],
    'gamma':[0,0.25,1.0],
    'reg_lambda':[0,1.0,10.0,50],
    'scale_pos_weight':[1,3,5]
    }
xgb_grid=GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="f1", n_jobs=-1, verbose = 2, cv=3)
xgb_grid.fit(X_rus, y_rus)